In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import fiona
import pyproj
import shapely.geometry as geom

Import the `nyc_all_gyms.csv` data we obtained by google maps API.

In [8]:
df_gyms = pd.read_csv('all_nyc_gyms.csv')
df_gyms

,formatted_address,geometry.location.lat,geometry.location.lng,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,icon,id,name,...,plus_code.compound_code,plus_code.global_code,price_level,rating,reference,types,user_ratings_total,state,zipcode,borough
0,"298 W 231st St, The Bronx, NY 10463, United St...",40.880539,-73.908334,40.881771,-73.907093,40.879072,-73.909793,https://maps.gstatic.com/mapfiles/place_api/ic...,a3485e7994aa8626891015b990080e6dc9d7b6b3,24 Hour Fitness,...,"V3JR+6M The Bronx, New York",87G8V3JR+6M,NaN,3.4,ChIJEfg3t5XzwokRtR2xHxFYi0Q,"['gym', 'health', 'point_of_interest', 'establ...",187.0,NY,10463,Bronx
1,"5520 Broadway, The Bronx, NY 10463, United States",40.877351,-73.905654,40.878701,-73.904305,40.876001,-73.907004,https://maps.gstatic.com/mapfiles/place_api/ic...,81389fd5d6b5dd805551b9beb642637143f06a5c,Blink Fitness,...,"V3GV+WP The Bronx, New York",87G8V3GV+WP,NaN,3.8,ChIJKftkWJHzwokRWTI8LcCn9LI,"['gym', 'health', 'point_of_interest', 'establ...",211.0,NY,10463,Bronx
2,"5500 Broadway, The Bronx, NY 10463, United States",40.876568,-73.906144,40.877848,-73.904852,40.875149,-73.907552,https://maps.gstatic.com/mapfiles/place_api/ic...,93a5f9fa33efd28fa5726eedeaae0a52803eb091,Astral Fitness,...,"V3GV+JG The Bronx, New York",87G8V3GV+JG,NaN,4.2,ChIJqczPWpHzwokR3qB2P5FDkIU,"['gym', 'health', 'point_of_interest', 'establ...",31.0,NY,10463,Bronx
3,"82 W 225th St, The Bronx, NY 10463, United States",40.874076,-73.909262,40.875480,-73.907841,40.872780,-73.910541,https://maps.gstatic.com/mapfiles/place_api/ic...,912d8b13c868012e545577eeaa8be0762506adfb,Planet Fitness,...,"V3FR+J7 The Bronx, New York",87G8V3FR+J7,NaN,4.2,ChIJr3JBXpLzwokR_dIaA6b0vd8,"['gym', 'spa', 'health', 'point_of_interest', ...",309.0,NY,10463,Bronx
4,"453 W 238th St, The Bronx, NY 10463, United St...",40.887410,-73.905057,40.888704,-73.903729,40.886005,-73.906429,https://maps.gstatic.com/mapfiles/place_api/ic...,a272c6a36fa158e12f45a31e7a2bc01896e26c8c,CrossFit La Caja,...,"V3PV+XX The Bronx, New York",87G8V3PV+XX,NaN,4.9,ChIJIUCs9LvzwokRLW1jjLy1ILQ,"['gym', 'health', 'point_of_interest', 'establ...",14.0,NY,10463,Bronx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1809,"385 Wild Ave, Staten Island, NY 10314, United ...",40.586235,-74.194648,40.587605,-74.193419,40.584905,-74.196118,https://maps.gstatic.com/mapfiles/place_api/ic...,c7127aa3afe2ca6831d948f3a33c9646fdf7d198,No Limitz Gymnastics and Cheerleading Center,...,HRP4+F4 New York,87G7HRP4+F4,NaN,4.4,ChIJncMQ419LwokRUFvqIcE7qVM,"['gym', 'health', 'point_of_interest', 'establ...",22.0,NY,10314,Staten_Island
1810,"1515 Richmond Ave, Staten Island, NY 10314, Un...",40.610805,-74.159848,40.612149,-74.158477,40.609449,-74.161177,https://maps.gstatic.com/mapfiles/place_api/ic...,3847e474ffd69080ef8fefb22aab119055f6ab74,Caddell dry dock,...,JR6R+83 New York,87G7JR6R+83,NaN,0.0,ChIJKeETt1pMwokR8n4kRFXJMes,"['gym', 'health', 'point_of_interest', 'establ...",0.0,NY,10314,Staten_Island
1811,"Commerce St, Staten Island, NY 10314, United S...",40.611892,-74.168454,40.613244,-74.167097,40.610544,-74.169796,https://maps.gstatic.com/mapfiles/place_api/ic...,2636ec1317fcdc6e6885e7c7f22ff40d32573c3f,Aeeee,...,JR6J+QJ New York,87G7JR6J+QJ,NaN,0.0,ChIJu7xtNlhMwokRiUrABb1gNSU,"['gym', 'health', 'point_of_interest', 'establ...",0.0,NY,10314,Staten_Island
1812,"Staten Island, NY 10314, United States",40.604036,-74.148965,40.605591,-74.147614,40.602891,-74.150314,https://maps.gstatic.com/mapfiles/place_api/ic...,13d703d00bcb59b837aade4e76d55d12da9512a0,College of Staten Island Sports and Recreation...,...,JV32+JC New York,87G7JV32+JC,NaN,4.6,ChIJ4cvV9u1NwokRGNvXDSeIOWY,"['gym', 'health', 'point_of_interest', 'establ...",14.0,NY,10314,Staten_Island


Convert the gym's lat, long locations as points and into a geopandas dataframe.

In [9]:
# convert lat, long as points
geometry = [geom.Point(xy) for xy in zip(df_gyms['geometry.location.lng'], 
                                        df_gyms['geometry.location.lat'])]
# drop lat, lng from csv.
df_gyms = df_gyms.drop(['geometry.location.lng', 
                        'geometry.location.lat'], axis=1)

# convert df and points into a geopandas df with espg:4326 
crs = {'init': 'epsg:4326'}
gdf_gyms = gpd.GeoDataFrame(df_gyms, crs=crs, geometry=geometry)

In [12]:
gdf_gyms.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1814 entries, 0 to 1813
Data columns (total 22 columns):
formatted_address                  1814 non-null object
geometry.viewport.northeast.lat    1814 non-null float64
geometry.viewport.northeast.lng    1814 non-null float64
geometry.viewport.southwest.lat    1814 non-null float64
geometry.viewport.southwest.lng    1814 non-null float64
icon                               1814 non-null object
id                                 1814 non-null object
name                               1814 non-null object
opening_hours.open_now             1442 non-null object
photos                             1447 non-null object
place_id                           1814 non-null object
plus_code.compound_code            1810 non-null object
plus_code.global_code              1810 non-null object
price_level                        30 non-null float64
rating                             1806 non-null float64
reference                          1814 n

In [11]:
gdf_gyms['geometry']

0       POINT (-73.90833 40.88054)
1       POINT (-73.90565 40.87735)
2       POINT (-73.90614 40.87657)
3       POINT (-73.90926 40.87408)
4       POINT (-73.90506 40.88741)
                   ...            
1809    POINT (-74.19465 40.58623)
1810    POINT (-74.15985 40.61080)
1811    POINT (-74.16845 40.61189)
1812    POINT (-74.14896 40.60404)
1813    POINT (-74.12319 40.63398)
Name: geometry, Length: 1814, dtype: geometry

Now let's export this a geoJSON

In [13]:
gdf_gyms.to_file("nyc_gyms.geojson", driver='GeoJSON')


### Let's also try to create an aggregate version by CD
We will count the number of gyms by CD.

In [5]:
# import CD shape file.
# project geometries to EPSG 4326 to match the projection of the obesity map
df_cd = gpd.read_file('final_data/Community Districts/geo_export_bf9282a4-4d98-4f1a-9606-0bf283c4c69d.shp').to_crs(fiona.crs.from_epsg(4326))
df_cd

,boro_cd,shape_area,shape_leng,geometry
0,311.0,1.031778e+08,51549.557899,"POLYGON ((-73.97299 40.60881, -73.97259 40.606..."
1,313.0,8.819569e+07,65821.875617,"POLYGON ((-73.98372 40.59582, -73.98305 40.595..."
2,312.0,9.952550e+07,52245.830495,"POLYGON ((-73.97140 40.64826, -73.97121 40.647..."
3,304.0,5.666322e+07,37008.100320,"POLYGON ((-73.89647 40.68234, -73.89653 40.682..."
4,206.0,4.266431e+07,35875.710998,"POLYGON ((-73.87185 40.84376, -73.87192 40.843..."
...,...,...,...,...
66,227.0,3.143201e+07,28391.629705,"POLYGON ((-73.87054 40.86967, -73.87053 40.869..."
67,401.0,1.715489e+08,90042.718108,"MULTIPOLYGON (((-73.90647 40.79018, -73.90251 ..."
68,402.0,1.398915e+08,71543.044665,"POLYGON ((-73.89792 40.75424, -73.89797 40.754..."
69,502.0,5.931981e+08,142669.724480,"MULTIPOLYGON (((-74.07347 40.57839, -74.07345 ..."


In [12]:
df_gyms = gpd.read_file('final_data/nyc_gyms.geojson').to_crs(fiona.crs.from_epsg(4326))
df_gyms

,formatted_address,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,icon,id,name,opening_hours.open_now,photos,...,plus_code.compound_code,plus_code.global_code,price_level,rating,reference,user_ratings_total,state,zipcode,borough,geometry
0,"298 W 231st St, The Bronx, NY 10463, United St...",40.881771,-73.907093,40.879072,-73.909793,https://maps.gstatic.com/mapfiles/place_api/ic...,a3485e7994aa8626891015b990080e6dc9d7b6b3,24 Hour Fitness,1,"[ { ""height"": 3264, ""html_attributions"": [ ""<a...",...,"V3JR+6M The Bronx, New York",87G8V3JR+6M,NaN,3.4,ChIJEfg3t5XzwokRtR2xHxFYi0Q,187.0,NY,10463,Bronx,POINT (-73.90833 40.88054)
1,"5520 Broadway, The Bronx, NY 10463, United States",40.878701,-73.904305,40.876001,-73.907004,https://maps.gstatic.com/mapfiles/place_api/ic...,81389fd5d6b5dd805551b9beb642637143f06a5c,Blink Fitness,1,"[ { ""height"": 4160, ""html_attributions"": [ ""<a...",...,"V3GV+WP The Bronx, New York",87G8V3GV+WP,NaN,3.8,ChIJKftkWJHzwokRWTI8LcCn9LI,211.0,NY,10463,Bronx,POINT (-73.90565 40.87735)
2,"5500 Broadway, The Bronx, NY 10463, United States",40.877848,-73.904852,40.875149,-73.907552,https://maps.gstatic.com/mapfiles/place_api/ic...,93a5f9fa33efd28fa5726eedeaae0a52803eb091,Astral Fitness,1,"[ { ""height"": 864, ""html_attributions"": [ ""<a ...",...,"V3GV+JG The Bronx, New York",87G8V3GV+JG,NaN,4.2,ChIJqczPWpHzwokR3qB2P5FDkIU,31.0,NY,10463,Bronx,POINT (-73.90614 40.87657)
3,"82 W 225th St, The Bronx, NY 10463, United States",40.875480,-73.907841,40.872780,-73.910541,https://maps.gstatic.com/mapfiles/place_api/ic...,912d8b13c868012e545577eeaa8be0762506adfb,Planet Fitness,1,"[ { ""height"": 4032, ""html_attributions"": [ ""<a...",...,"V3FR+J7 The Bronx, New York",87G8V3FR+J7,NaN,4.2,ChIJr3JBXpLzwokR_dIaA6b0vd8,309.0,NY,10463,Bronx,POINT (-73.90926 40.87408)
4,"453 W 238th St, The Bronx, NY 10463, United St...",40.888704,-73.903729,40.886005,-73.906429,https://maps.gstatic.com/mapfiles/place_api/ic...,a272c6a36fa158e12f45a31e7a2bc01896e26c8c,CrossFit La Caja,1,"[ { ""height"": 1334, ""html_attributions"": [ ""<a...",...,"V3PV+XX The Bronx, New York",87G8V3PV+XX,NaN,4.9,ChIJIUCs9LvzwokRLW1jjLy1ILQ,14.0,NY,10463,Bronx,POINT (-73.90506 40.88741)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1809,"385 Wild Ave, Staten Island, NY 10314, United ...",40.587605,-74.193419,40.584905,-74.196118,https://maps.gstatic.com/mapfiles/place_api/ic...,c7127aa3afe2ca6831d948f3a33c9646fdf7d198,No Limitz Gymnastics and Cheerleading Center,1,"[ { ""height"": 4032, ""html_attributions"": [ ""<a...",...,HRP4+F4 New York,87G7HRP4+F4,NaN,4.4,ChIJncMQ419LwokRUFvqIcE7qVM,22.0,NY,10314,Staten_Island,POINT (-74.19465 40.58623)
1810,"1515 Richmond Ave, Staten Island, NY 10314, Un...",40.612149,-74.158477,40.609449,-74.161177,https://maps.gstatic.com/mapfiles/place_api/ic...,3847e474ffd69080ef8fefb22aab119055f6ab74,Caddell dry dock,None,"[ { ""height"": 1836, ""html_attributions"": [ ""<a...",...,JR6R+83 New York,87G7JR6R+83,NaN,0.0,ChIJKeETt1pMwokR8n4kRFXJMes,0.0,NY,10314,Staten_Island,POINT (-74.15985 40.61080)
1811,"Commerce St, Staten Island, NY 10314, United S...",40.613244,-74.167097,40.610544,-74.169796,https://maps.gstatic.com/mapfiles/place_api/ic...,2636ec1317fcdc6e6885e7c7f22ff40d32573c3f,Aeeee,None,None,...,JR6J+QJ New York,87G7JR6J+QJ,NaN,0.0,ChIJu7xtNlhMwokRiUrABb1gNSU,0.0,NY,10314,Staten_Island,POINT (-74.16845 40.61189)
1812,"Staten Island, NY 10314, United States",40.605591,-74.147614,40.602891,-74.150314,https://maps.gstatic.com/mapfiles/place_api/ic...,13d703d00bcb59b837aade4e76d55d12da9512a0,College of Staten Island Sports and Recreation...,None,"[ { ""height"": 4032, ""html_attributions"": [ ""<a...",...,JV32+JC New York,87G7JV32+JC,NaN,4.6,ChIJ4cvV9u1NwokRGNvXDSeIOWY,14.0,NY,10314,Staten_Island,POINT (-74.14896 40.60404)


Spatial join gym locations to CD polygons

In [14]:
df_sjoin = gpd.sjoin(df_cd, df_gyms, how='inner', op='intersects')
df_sjoin

,boro_cd,shape_area,shape_leng,geometry,index_right,formatted_address,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,geometry.viewport.southwest.lat,geometry.viewport.southwest.lng,...,place_id,plus_code.compound_code,plus_code.global_code,price_level,rating,reference,user_ratings_total,state,zipcode,borough
0,311.0,1.031778e+08,51549.557899,"POLYGON ((-73.97299 40.60881, -73.97259 40.606...",707,"2266 McDonald Ave, Brooklyn, NY 11223, United ...",40.598547,-73.971966,40.595848,-73.974666,...,ChIJUeBXWvhEwokRQEVVYMA3Drg,"H2WG+VJ Brooklyn, New York",87G8H2WG+VJ,NaN,5.0,ChIJUeBXWvhEwokRQEVVYMA3Drg,7.0,NY,11223,Brooklyn
0,311.0,1.031778e+08,51549.557899,"POLYGON ((-73.97299 40.60881, -73.97259 40.606...",439,"2402 86th St, Brooklyn, NY 11214, United States",40.600216,-73.987562,40.597516,-73.990261,...,ChIJ21LmNQdFwokR35mm2GqLKIg,"H2X6+GC Brooklyn, New York",87G8H2X6+GC,NaN,3.7,ChIJ21LmNQdFwokR35mm2GqLKIg,247.0,NY,11214,Brooklyn
0,311.0,1.031778e+08,51549.557899,"POLYGON ((-73.97299 40.60881, -73.97259 40.606...",483,"215 Avenue T, Brooklyn, NY 11223, United States",40.600269,-73.975715,40.597569,-73.978415,...,ChIJXYCq0v5EwokRa5gA_c8AvqQ,"H2XF+H5 Brooklyn, New York",87G8H2XF+H5,NaN,5.0,ChIJXYCq0v5EwokRa5gA_c8AvqQ,9.0,NY,11223,Brooklyn
0,311.0,1.031778e+08,51549.557899,"POLYGON ((-73.97299 40.60881, -73.97259 40.606...",706,"289 Avenue S, Brooklyn, NY 11223, United States",40.602974,-73.973326,40.600274,-73.976026,...,ChIJn9SwyftEwokRluVufIeNDW8,J22G+M4 New York,87G8J22G+M4,NaN,0.0,ChIJn9SwyftEwokRluVufIeNDW8,0.0,NY,11223,Brooklyn
0,311.0,1.031778e+08,51549.557899,"POLYGON ((-73.97299 40.60881, -73.97259 40.606...",705,"1837 Stillwell Ave, Brooklyn, NY 11223, United...",40.603079,-73.984742,40.600380,-73.987442,...,ChIJB-ekeQFFwokRrvhlv_9kHQE,"J227+MJ Brooklyn, New York",87G8J227+MJ,NaN,5.0,ChIJB-ekeQFFwokRrvhlv_9kHQE,10.0,NY,11223,Brooklyn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,502.0,5.931981e+08,142669.724480,"MULTIPOLYGON (((-74.07347 40.57839, -74.07345 ...",1733,"501 Brielle Ave, Staten Island, NY 10314, Unit...",40.593105,-74.137778,40.590405,-74.140477,...,ChIJk4yuBxFMwokRQ7fotgXjEg8,"HVR6+P8 Staten Island, New York",87G7HVR6+P8,NaN,4.3,ChIJk4yuBxFMwokRQ7fotgXjEg8,42.0,NY,10314,Staten_Island
69,502.0,5.931981e+08,142669.724480,"MULTIPOLYGON (((-74.07347 40.57839, -74.07345 ...",1740,"333 Sand Ln, Staten Island, NY 10305, United S...",40.593196,-74.066081,40.590496,-74.068780,...,ChIJP8cnrhlPwokRuGMOYPdn1FU,"HWRM+P2 Staten Island, New York",87G7HWRM+P2,NaN,5.0,ChIJP8cnrhlPwokRuGMOYPdn1FU,17.0,NY,10305,Staten_Island
69,502.0,5.931981e+08,142669.724480,"MULTIPOLYGON (((-74.07347 40.57839, -74.07345 ...",1751,"1466 Manor Rd, Staten Island, NY 10314, United...",40.596345,-74.125048,40.593645,-74.127748,...,ChIJe_YFyh9MwokRdNHd3Q4dvbA,"HVVF+WC Staten Island, New York",87G7HVVF+WC,NaN,3.9,ChIJe_YFyh9MwokRdNHd3Q4dvbA,39.0,NY,10314,Staten_Island
70,595.0,5.471514e+07,110237.328244,"MULTIPOLYGON (((-74.11834 40.55046, -74.11676 ...",1725,"204 Maloney Dr, Staten Island, NY 10305, Unite...",40.605387,-74.061525,40.602687,-74.064224,...,ChIJRxUr2whPwokR7GK7zs84ZZk,JW3P+HQ New York,87G7JW3P+HQ,NaN,4.3,ChIJRxUr2whPwokR7GK7zs84ZZk,12.0,NY,10305,Staten_Island


Looks like 1811/1814 of the gyms were in the community districts


In [19]:
# see how many boro_cds had gyms
df_sjoin['boro_cd'].nunique()

63

Looks like 63 of the CDs had gyms inside them, which makes sense since there are number of CDs that are in inhabitable places, like airports, etc

In [23]:
# count number of gyms by community district 
cd_counts = df_sjoin.pivot_table(index=['boro_cd'], aggfunc={'place_id':'nunique'}).reset_index()
cd_counts

,boro_cd,place_id
0,101.0,73
1,102.0,77
2,103.0,47
3,104.0,56
4,105.0,165
...,...,...
58,482.0,1
59,501.0,28
60,502.0,47
61,503.0,36


In [27]:
# merge back to the origin cd geometries
df_gym_cd = df_cd.merge(cd_counts).rename(columns={'place_id':'number_of_gyms'})
df_gym_cd

,boro_cd,shape_area,shape_leng,geometry,number_of_gyms
0,311.0,1.031778e+08,51549.557899,"POLYGON ((-73.97299 40.60881, -73.97259 40.606...",20
1,313.0,8.819569e+07,65821.875617,"POLYGON ((-73.98372 40.59582, -73.98305 40.595...",14
2,312.0,9.952550e+07,52245.830495,"POLYGON ((-73.97140 40.64826, -73.97121 40.647...",8
3,304.0,5.666322e+07,37008.100320,"POLYGON ((-73.89647 40.68234, -73.89653 40.682...",13
4,206.0,4.266431e+07,35875.710998,"POLYGON ((-73.87185 40.84376, -73.87192 40.843...",2
...,...,...,...,...,...
58,211.0,1.003136e+08,45740.919535,"POLYGON ((-73.82834 40.86089, -73.82843 40.860...",15
59,401.0,1.715489e+08,90042.718108,"MULTIPOLYGON (((-73.90647 40.79018, -73.90251 ...",69
60,402.0,1.398915e+08,71543.044665,"POLYGON ((-73.89792 40.75424, -73.89797 40.754...",35
61,502.0,5.931981e+08,142669.724480,"MULTIPOLYGON (((-74.07347 40.57839, -74.07345 ...",47


Export as a geojson for heatmap viz.


In [28]:
df_gym_cd.to_file("final_data/gyms_cd.geojson", driver='GeoJSON')
